In [72]:
import pandas as pd
import numpy as np
import duckdb

In [73]:
import warnings
warnings.filterwarnings('ignore')

In [74]:
data_folder = "staticNotDeployed/"
static_folder = "static/"

In [75]:
# scenario_BNZ_path = "simulations_new/BNZ.csv"
# scenario_holder_path = "simulations_new/BNZ.csv"

# New data march 2025
# scenario_BNZ_path = data_folder + "simulations_new/march2025/BNZ_£millions_annualy.csv"
# socio_factors_path = "UK_Archetypes_global_measures.csv"

# april 2025
scenario_BNZ_path = data_folder + "simulations_new/april2025/cleaned_total_outputs_millions.csv"
socio_factors_path = data_folder + "simulations_new/april2025/SEF.csv"

## Create pandas tables

In [177]:
# df_socio = pd.read_csv(static_folder+socio_factors_path)
df_socio = pd.read_csv(socio_factors_path)

In [178]:
# Remove one row on NaN
df_socio = df_socio.dropna()

In [179]:
df_socio = df_socio.convert_dtypes()

In [180]:
df_socio

,Nation,Region,LAD,LSOA.DZ.CD,LSOA.DZ.NM,Households,Population,Under.35,Over.65,EPC,...,Rurality,House.value,Fuel.Type,Fuel.consumption.total,Floor.area,Gas.flag,Number.cars,Urban.trips,Total.vkm,Urban.vkm
0,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,249,738,0.45664,0.154472,4,...,0.0,137687.9444,3,37665.0,180.0,0,2,414.15324,12922.63505,2584.316217
1,NI,Northern Ireland,N09000001,N20000002,Dunsilly_B1,127,331,0.372372,0.204204,4,...,0.0,149950.0,3,35381.25,166.5,0,2,375.943839,11730.40451,2345.889557
2,NI,Northern Ireland,N09000001,N20000003,Dunsilly_A2,128,365,0.442623,0.229508,4,...,0.5,137687.9444,3,32461.0,143.0,0,2,368.684492,11503.89439,2300.591227
3,NI,Northern Ireland,N09000001,N20000004,Dunsilly_A3,219,663,0.470588,0.176471,5,...,0.5,137687.9444,3,30303.0,126.0,0,2,386.977279,12074.67589,2414.738218
4,NI,Northern Ireland,N09000001,N20000005,Dunsilly_B2,231,676,0.446746,0.189349,4,...,0.0,134250.0,3,35165.625,181.5,0,2,407.61788,12718.71519,2543.535571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46421,Scotland,Scotland,S12000040,S01013477,Broxburn South - 06,311,644,0.34,0.23,3,...,0.9,175000.0,1,31622.0,97.0,1,2,391.755872,15827.26805,10490.75574
46422,Scotland,Scotland,S12000040,S01013478,Broxburn East - 01,352,823,0.42,0.14,3,...,0.9,130000.0,1,12784.0,68.0,1,2,699.209847,15223.57237,10090.60937
46423,Scotland,Scotland,S12000040,S01013479,Broxburn East - 02,445,734,0.34,0.26,3,...,0.9,72500.0,1,15801.0,69.0,1,2,461.180963,14259.49696,9451.593232
46424,Scotland,Scotland,S12000040,S01013480,Broxburn East - 03,303,703,0.4,0.23,3,...,0.9,72500.0,1,17025.0,75.0,1,2,359.522793,13406.18716,8885.995642


In [181]:
df_socio.rename(columns=lambda x: x.replace('.', '_'), inplace=True)

In [182]:
df_socio.columns

Index(['Nation', 'Region', 'LAD', 'LSOA_DZ_CD', 'LSOA_DZ_NM', 'Households',
       'Population', 'Under_35', 'Over_65', 'EPC', 'Median_Income', 'Tenure',
       'Typology', 'Unemployment', 'Rurality', 'House_value', 'Fuel_Type',
       'Fuel_consumption_total', 'Floor_area', 'Gas_flag', 'Number_cars',
       'Urban_trips', 'Total_vkm', 'Urban_vkm'],
      dtype='object')

In [183]:
# EPC and gas_flag have weird values
df_socio.dtypes

Nation                    string[python]
Region                    string[python]
LAD                       string[python]
LSOA_DZ_CD                string[python]
LSOA_DZ_NM                string[python]
Households                         Int64
Population                         Int64
Under_35                         Float64
Over_65                          Float64
EPC                               object
Median_Income                    Float64
Tenure                             Int64
Typology                           Int64
Unemployment                     Float64
Rurality                         Float64
House_value                      Float64
Fuel_Type                          Int64
Fuel_consumption_total           Float64
Floor_area                       Float64
Gas_flag                           Int64
Number_cars                        Int64
Urban_trips                      Float64
Total_vkm                        Float64
Urban_vkm                        Float64
dtype: object

In [184]:
set(df_socio.EPC)

{1, 2, '2', 3, '3', 4, '4', 5, '5', 6, '6', 7, '7', 'd', 'e'}

In [185]:
set(df_socio.Gas_flag)

{0, 1}

In [186]:
# Replace the string 'unknown' with pd.NA
df_socio['Gas_flag'] = df_socio['Gas_flag'].replace('Y', pd.NA)

# Step 2: Convert to numeric (converts '1.0' to 1.0 as float)
df_socio['Gas_flag'] = pd.to_numeric(df_socio['Gas_flag'], errors='coerce')

df_socio['Gas_flag'] = df_socio['Gas_flag'].astype('Int16')

In [187]:
# Replace the string 'unknown' with pd.NA
df_socio['EPC'] = df_socio['EPC'].replace('d', pd.NA)

# Step 2: Convert to numeric (converts '1.0' to 1.0 as float)
df_socio['EPC'] = pd.to_numeric(df_socio['EPC'], errors='coerce')

df_socio['EPC'] = df_socio['EPC'].astype('Int16')

#### Add LADs to the SEF table from the lookup LAD tables

In [188]:
# df_lad_NI = pd.read_csv("static/LAD/NI_DZ_LAD.csv")
df_lad_Eng = pd.read_csv("static/LAD/Eng_wales_LSOA_LADs.csv")
# df_lad_Scotland = pd.read_csv("static/LAD/Scotland_DZ_LA.csv", encoding='latin1')

In [189]:
df_lad_Eng.head()

,LSOA11CD,LSOA11NM,LSOA21CD,LSOA21NM,CHGIND,LAD22CD,LAD22NM,LAD22NMW,ObjectId
0,E01000001,City of London 001A,E01000001,City of London 001A,U,E09000001,City of London,NaN,1
1,E01000002,City of London 001B,E01000002,City of London 001B,U,E09000001,City of London,NaN,2
2,E01000003,City of London 001C,E01000003,City of London 001C,U,E09000001,City of London,NaN,3
3,E01000005,City of London 001E,E01000005,City of London 001E,U,E09000001,City of London,NaN,4
4,E01000006,Barking and Dagenham 016A,E01000006,Barking and Dagenham 016A,U,E09000002,Barking and Dagenham,NaN,5


In [190]:
df_socio

,Nation,Region,LAD,LSOA_DZ_CD,LSOA_DZ_NM,Households,Population,Under_35,Over_65,EPC,...,Rurality,House_value,Fuel_Type,Fuel_consumption_total,Floor_area,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm
0,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,249,738,0.45664,0.154472,4,...,0.0,137687.9444,3,37665.0,180.0,0,2,414.15324,12922.63505,2584.316217
1,NI,Northern Ireland,N09000001,N20000002,Dunsilly_B1,127,331,0.372372,0.204204,4,...,0.0,149950.0,3,35381.25,166.5,0,2,375.943839,11730.40451,2345.889557
2,NI,Northern Ireland,N09000001,N20000003,Dunsilly_A2,128,365,0.442623,0.229508,4,...,0.5,137687.9444,3,32461.0,143.0,0,2,368.684492,11503.89439,2300.591227
3,NI,Northern Ireland,N09000001,N20000004,Dunsilly_A3,219,663,0.470588,0.176471,5,...,0.5,137687.9444,3,30303.0,126.0,0,2,386.977279,12074.67589,2414.738218
4,NI,Northern Ireland,N09000001,N20000005,Dunsilly_B2,231,676,0.446746,0.189349,4,...,0.0,134250.0,3,35165.625,181.5,0,2,407.61788,12718.71519,2543.535571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46421,Scotland,Scotland,S12000040,S01013477,Broxburn South - 06,311,644,0.34,0.23,3,...,0.9,175000.0,1,31622.0,97.0,1,2,391.755872,15827.26805,10490.75574
46422,Scotland,Scotland,S12000040,S01013478,Broxburn East - 01,352,823,0.42,0.14,3,...,0.9,130000.0,1,12784.0,68.0,1,2,699.209847,15223.57237,10090.60937
46423,Scotland,Scotland,S12000040,S01013479,Broxburn East - 02,445,734,0.34,0.26,3,...,0.9,72500.0,1,15801.0,69.0,1,2,461.180963,14259.49696,9451.593232
46424,Scotland,Scotland,S12000040,S01013480,Broxburn East - 03,303,703,0.4,0.23,3,...,0.9,72500.0,1,17025.0,75.0,1,2,359.522793,13406.18716,8885.995642


In [191]:
# for row in df[df.Nation == "Eng/Wales"].iterrows():
#     row.Look

    
# LAD22CD
# df['LAD'] = df['Lookup_Value'].map(df_lad_Eng.set_index('LSOA11CD')['LAD22CD']).fillna(df['LAD'])

# Create a lookup dictionary from df2
lookup = dict(zip(df_lad_Eng['LSOA11CD'], df_lad_Eng['LAD22CD']))
lookup2 = dict(zip(df_lad_Eng['LSOA21CD'], df_lad_Eng['LAD22CD']))


# Loop through df and update status if there's a match
for idx, row in df_socio[df_socio.Nation == "Eng/Wales"].iterrows():
    lsoa = row['LSOA_DZ_CD']
    if lsoa in lookup:
        df_socio.at[idx, 'LAD'] = lookup[lsoa]
    # if lsoa in lookup2:
        # df.at[idx, 'LAD'] = lookup[lsoa]

In [192]:
#df_socio = pd.merge(df_socio, df_lad_Eng[["LSOA"]], left_on='Lookup_Value', right_on='LSOA_DZ_CD', how='inner')

### Cobenefit table

In [193]:
df = pd.read_csv(scenario_BNZ_path)

In [194]:
# Step: Drop rows where any column contains the value '#DIV/0!'
# There are rows with these weird values in the new dataset
df = df[~df.isin(['#DIV/0!']).any(axis=1)]

In [195]:
# To not change the queries in the app
df["scenario"] = "BNZ"

In [196]:
df.dtypes

Lookup.Value      object
Coben             object
2025             float64
2026             float64
2027             float64
2028             float64
2029             float64
2030             float64
2031             float64
2032             float64
2033             float64
2034             float64
2035             float64
2036             float64
2037             float64
2038             float64
2039             float64
2040             float64
2041             float64
2042             float64
2043             float64
2044             float64
2045             float64
2046             float64
2047             float64
2048             float64
2049             float64
2050             float64
Sum_2025_2050    float64
scenario          object
dtype: object

In [197]:
# Convert columns with names from 2025 to 2050 to float
year_columns = [str(year) for year in range(2025, 2051)]
df[year_columns] = df[year_columns].astype(np.float32)

In [198]:
df.dtypes

Lookup.Value      object
Coben             object
2025             float32
2026             float32
2027             float32
2028             float32
2029             float32
2030             float32
2031             float32
2032             float32
2033             float32
2034             float32
2035             float32
2036             float32
2037             float32
2038             float32
2039             float32
2040             float32
2041             float32
2042             float32
2043             float32
2044             float32
2045             float32
2046             float32
2047             float32
2048             float32
2049             float32
2050             float32
Sum_2025_2050    float64
scenario          object
dtype: object

In [199]:
# Create total column
# df["total (£m)"] = df[ [ f'{i} (£m)' for i in range(2025, 2051)]].sum(axis=1)
df["total (£m)"] = df[ [ f'{i}' for i in range(2025, 2051)]].sum(axis=1)

In [200]:
# Rename columns so it does not contain special characters (not needed anymore I think)
df.columns = df.columns.str.replace(' (£m)', '')

In [201]:
# Rename columns: replace spaces and points with underscores
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('.', '_')

In [202]:
df.head()

,Lookup_Value,Coben,2025,2026,2027,2028,2029,2030,2031,2032,...,2044,2045,2046,2047,2048,2049,2050,Sum_2025_2050,scenario,total
0,N20000001,Air quality,0.004233,0.005133,0.006093,0.007120,0.007986,0.008998,0.009932,0.011715,...,0.032759,0.033621,0.034118,0.034120,0.033862,0.033590,0.033330,0.553835,BNZ,0.553835
1,N20000001,Noise,0.000000,0.000000,0.001673,0.001686,0.001527,0.002845,0.002816,0.003854,...,0.022019,0.020987,0.021152,0.020541,0.020048,0.009599,0.009366,0.289893,BNZ,0.289893
2,N20000001,Congestion,0.010883,0.018770,0.024767,0.029034,0.031753,0.038121,0.034502,0.030951,...,-0.026544,-0.031189,-0.035330,-0.039679,-0.044307,-0.048308,-0.058733,-0.038290,BNZ,-0.038290
3,N20000001,Road repairs,0.000216,0.000266,0.000290,0.000288,0.000351,0.000409,0.000350,0.000293,...,-0.000639,-0.000726,-0.000774,-0.000837,-0.000903,-0.000959,-0.001097,-0.005605,BNZ,-0.005605
4,N20000001,Road safety,0.003759,0.006488,0.008707,0.010389,0.011580,0.013288,0.012212,0.011142,...,-0.011025,-0.013359,-0.015101,-0.017278,-0.019630,-0.021800,-0.026205,-0.040553,BNZ,-0.040553


In [203]:
df.total

0         0.553835
1         0.289893
2        -0.038290
3        -0.005605
4        -0.040553
            ...   
557107    0.176891
557108    0.000002
557109    0.005054
557110    0.069160
557111    2.930622
Name: total, Length: 557112, dtype: float32

In [204]:
np.max(df.total)

130.3221435546875

In [205]:
np.mean(df.total)

0.5207034

In [206]:
df.dtypes

Lookup_Value      object
Coben             object
2025             float32
2026             float32
2027             float32
2028             float32
2029             float32
2030             float32
2031             float32
2032             float32
2033             float32
2034             float32
2035             float32
2036             float32
2037             float32
2038             float32
2039             float32
2040             float32
2041             float32
2042             float32
2043             float32
2044             float32
2045             float32
2046             float32
2047             float32
2048             float32
2049             float32
2050             float32
Sum_2025_2050    float64
scenario          object
total            float32
dtype: object

## Join SE factors and cobenefs

In [207]:
df = pd.merge(df, df_socio, left_on='Lookup_Value', right_on='LSOA_DZ_CD', how='left')

In [208]:
df.head()

,Lookup_Value,Coben,2025,2026,2027,2028,2029,2030,2031,2032,...,Rurality,House_value,Fuel_Type,Fuel_consumption_total,Floor_area,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm
0,N20000001,Air quality,0.004233,0.005133,0.006093,0.007120,0.007986,0.008998,0.009932,0.011715,...,0.0,137687.9444,3,37665.0,180.0,0,2,414.15324,12922.63505,2584.316217
1,N20000001,Noise,0.000000,0.000000,0.001673,0.001686,0.001527,0.002845,0.002816,0.003854,...,0.0,137687.9444,3,37665.0,180.0,0,2,414.15324,12922.63505,2584.316217
2,N20000001,Congestion,0.010883,0.018770,0.024767,0.029034,0.031753,0.038121,0.034502,0.030951,...,0.0,137687.9444,3,37665.0,180.0,0,2,414.15324,12922.63505,2584.316217
3,N20000001,Road repairs,0.000216,0.000266,0.000290,0.000288,0.000351,0.000409,0.000350,0.000293,...,0.0,137687.9444,3,37665.0,180.0,0,2,414.15324,12922.63505,2584.316217
4,N20000001,Road safety,0.003759,0.006488,0.008707,0.010389,0.011580,0.013288,0.012212,0.011142,...,0.0,137687.9444,3,37665.0,180.0,0,2,414.15324,12922.63505,2584.316217


In [209]:
# Rename cobenef to always keep the same name (changed depending the version)
df.rename(columns={'Coben': 'co_benefit_type'}, inplace=True)

## Aggregating the time

In [210]:
# Number of years merging
time_step = 5

In [211]:
years = list(range( 2025, 2051 ))
#years

In [212]:
len(years)

26

In [213]:
df[["2025", "2026"]]

,2025,2026
0,0.004233,0.005133
1,0.000000,0.000000
2,0.010883,0.018770
3,0.000216,0.000266
4,0.003759,0.006488
...,...,...
557107,0.000000,0.000000
557108,0.000000,0.000000
557109,0.000000,0.000000
557110,0.000000,0.000000


In [214]:
#  AGGREGATE TIME: can disable
if True:
    for i in range(0, len(years) - ( time_step - 1), time_step):
        window_years = [str(year) for year in years[i:i+5]]
        print(window_years)
        window_sum = df[window_years].sum(axis=1)
        df[f'Y{window_years[0]}_{window_years[-1]}'] = window_sum
        # df[f'{window_years[0]}_{window_years[-1]}'] = window_sum

    # Delete single values columns for space
    df = df.drop(columns=[str(year) for year in years])

['2025', '2026', '2027', '2028', '2029']
['2030', '2031', '2032', '2033', '2034']
['2035', '2036', '2037', '2038', '2039']
['2040', '2041', '2042', '2043', '2044']
['2045', '2046', '2047', '2048', '2049']


In [215]:
# df[ ['2025', '2026', '2027', '2028', '2029', 'Y2025_2029'] ]

In [216]:
# CONVERT INT64 into int32
df[df.select_dtypes(np.int64).columns] = df.select_dtypes(np.int64).astype(np.int32)

In [217]:
df

,Lookup_Value,co_benefit_type,Sum_2025_2050,scenario,total,Nation,Region,LAD,LSOA_DZ_CD,LSOA_DZ_NM,...,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm,Y2025_2029,Y2030_2034,Y2035_2039,Y2040_2044,Y2045_2049
0,N20000001,Air quality,0.553835,BNZ,0.553835,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,3.056529e-02,6.008671e-02,1.090745e-01,1.514672e-01,1.693105e-01
1,N20000001,Noise,0.289893,BNZ,0.289893,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,4.884827e-03,2.451355e-02,5.925058e-02,9.955058e-02,9.232734e-02
2,N20000001,Congestion,-0.038290,BNZ,-0.038290,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,1.152073e-01,1.532783e-01,4.141028e-02,-9.063941e-02,-1.988135e-01
3,N20000001,Road repairs,-0.005605,BNZ,-0.005605,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,1.409627e-03,1.433914e-03,-5.476982e-04,-2.604994e-03,-4.198740e-03
4,N20000001,Road safety,-0.040553,BNZ,-0.040553,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,4.092452e-02,5.490646e-02,1.417326e-02,-3.718402e-02,-8.716834e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557107,S01013481,Excess cold,0.176891,BNZ,0.176891,Scotland,Scotland,S12000040,S01013481,Broxburn East - 04,...,1,2,297.536105,18642.74233,12356.9308,7.209255e-04,1.010945e-02,3.031537e-02,5.422976e-02,6.795486e-02
557108,S01013481,Excess heat,0.000002,BNZ,0.000002,Scotland,Scotland,S12000040,S01013481,Broxburn East - 04,...,1,2,297.536105,18642.74233,12356.9308,-7.092015e-09,2.617461e-08,3.140020e-07,7.091818e-07,7.697108e-07
557109,S01013481,Dampness,0.005054,BNZ,0.005054,Scotland,Scotland,S12000040,S01013481,Broxburn East - 04,...,1,2,297.536105,18642.74233,12356.9308,3.165534e-05,4.354297e-04,1.086547e-03,1.436800e-03,1.706385e-03
557110,S01013481,Diet change,0.069160,BNZ,0.069160,Scotland,Scotland,S12000040,S01013481,Broxburn East - 04,...,1,2,297.536105,18642.74233,12356.9308,9.789419e-03,1.568052e-02,1.467805e-02,1.372563e-02,1.282618e-02


In [218]:
set(df.EPC)

{1, 2, 3, 4, 5, 6, 7, <NA>}

In [219]:
df.dtypes

Lookup_Value                      object
co_benefit_type                   object
Sum_2025_2050                    float64
scenario                          object
total                            float32
Nation                    string[python]
Region                    string[python]
LAD                       string[python]
LSOA_DZ_CD                string[python]
LSOA_DZ_NM                string[python]
Households                         int32
Population                         int32
Under_35                         Float64
Over_65                          Float64
EPC                                Int16
Median_Income                    Float64
Tenure                             int32
Typology                           int32
Unemployment                     Float64
Rurality                         Float64
House_value                      Float64
Fuel_Type                          int32
Fuel_consumption_total           Float64
Floor_area                       Float64
Gas_flag        

In [220]:
df[(df["co_benefit_type"] == "Noise") & (df["Lookup_Value"] == "N20000001")]

,Lookup_Value,co_benefit_type,Sum_2025_2050,scenario,total,Nation,Region,LAD,LSOA_DZ_CD,LSOA_DZ_NM,...,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm,Y2025_2029,Y2030_2034,Y2035_2039,Y2040_2044,Y2045_2049
1,N20000001,Noise,0.289893,BNZ,0.289893,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,0.004885,0.024514,0.059251,0.099551,0.092327


In [221]:
df

,Lookup_Value,co_benefit_type,Sum_2025_2050,scenario,total,Nation,Region,LAD,LSOA_DZ_CD,LSOA_DZ_NM,...,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm,Y2025_2029,Y2030_2034,Y2035_2039,Y2040_2044,Y2045_2049
0,N20000001,Air quality,0.553835,BNZ,0.553835,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,3.056529e-02,6.008671e-02,1.090745e-01,1.514672e-01,1.693105e-01
1,N20000001,Noise,0.289893,BNZ,0.289893,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,4.884827e-03,2.451355e-02,5.925058e-02,9.955058e-02,9.232734e-02
2,N20000001,Congestion,-0.038290,BNZ,-0.038290,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,1.152073e-01,1.532783e-01,4.141028e-02,-9.063941e-02,-1.988135e-01
3,N20000001,Road repairs,-0.005605,BNZ,-0.005605,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,1.409627e-03,1.433914e-03,-5.476982e-04,-2.604994e-03,-4.198740e-03
4,N20000001,Road safety,-0.040553,BNZ,-0.040553,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,4.092452e-02,5.490646e-02,1.417326e-02,-3.718402e-02,-8.716834e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557107,S01013481,Excess cold,0.176891,BNZ,0.176891,Scotland,Scotland,S12000040,S01013481,Broxburn East - 04,...,1,2,297.536105,18642.74233,12356.9308,7.209255e-04,1.010945e-02,3.031537e-02,5.422976e-02,6.795486e-02
557108,S01013481,Excess heat,0.000002,BNZ,0.000002,Scotland,Scotland,S12000040,S01013481,Broxburn East - 04,...,1,2,297.536105,18642.74233,12356.9308,-7.092015e-09,2.617461e-08,3.140020e-07,7.091818e-07,7.697108e-07
557109,S01013481,Dampness,0.005054,BNZ,0.005054,Scotland,Scotland,S12000040,S01013481,Broxburn East - 04,...,1,2,297.536105,18642.74233,12356.9308,3.165534e-05,4.354297e-04,1.086547e-03,1.436800e-03,1.706385e-03
557110,S01013481,Diet change,0.069160,BNZ,0.069160,Scotland,Scotland,S12000040,S01013481,Broxburn East - 04,...,1,2,297.536105,18642.74233,12356.9308,9.789419e-03,1.568052e-02,1.467805e-02,1.372563e-02,1.282618e-02


## Add the number of households (we have the values in another old table)

In [120]:
df_nb_households = pd.read_csv(data_folder + "simulations_new/BNZ.csv")

In [74]:
len(df_nb_households)

557112

In [79]:
df_for_merge = df_nb_households[['Lookup_Value', 'HHs']]
df_for_merge = df_for_merge.drop_duplicates('Lookup_Value')

In [84]:
df = df.merge(df_nb_households[['Lookup_Value', 'HHs']].drop_duplicates('Lookup_Value'), on='Lookup_Value', how='inner')

In [87]:
len(df)

556122

## Export table as parquet file

In [222]:
df.to_parquet('static/database.parquet')

In [197]:
# Not needed anymore, everything is in the same table
# df_socio.to_parquet('static/tableSocio.parquet')

In [122]:
df.columns

Index(['Lookup_Value', 'co_benefit_type', 'Sum_2025_2050', 'scenario', 'total',
       'Nation', 'Region', 'LAD', 'LSOA_DZ_CD', 'LSOA_DZ_NM', 'Households',
       'Population', 'Under_35', 'Over_65', 'EPC', 'Median_Income', 'Tenure',
       'Typology', 'Unemployment', 'Rurality', 'House_value', 'Fuel_Type',
       'Fuel_consumption_total', 'Floor_area', 'Gas_flag', 'Number_cars',
       'Urban_trips', 'Total_vkm', 'Urban_vkm', 'Y2025_2029', 'Y2030_2034',
       'Y2035_2039', 'Y2040_2044', 'Y2045_2049'],
      dtype='object')

In [91]:
set(df.Nation)

{'Eng/Wales', 'NI', 'Scotland'}

In [133]:
df.co_benefit_type

0          Air quality
1                Noise
2           Congestion
3         Road repairs
4          Road safety
              ...     
556117     Excess cold
556118     Excess heat
556119        Dampness
556120     Diet change
556121           Total
Name: co_benefit_type, Length: 556122, dtype: object

## Export subset for faster development 

In [122]:
df

,Lookup_Value,co_benefit_type,2025,2026,2027,2028,2029,2030,2031,2032,...,Rurality,House_value,Fuel_Type,Fuel_consumption_total,Floor_area,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm
0,N20000001,Air quality,3.495246e-03,4.545631e-03,5.846987e-03,7.227724e-03,8.178627e-03,8.060945e-03,8.426776e-03,8.942410e-03,...,2,137688,3,37665,180,0,2,414,12923,2584
1,N20000001,Noise,1.345039e-03,2.122763e-03,3.191027e-03,3.107497e-03,3.972673e-03,6.072646e-03,6.876487e-03,6.949075e-03,...,2,137688,3,37665,180,0,2,414,12923,2584
2,N20000001,Congestion,2.332261e-03,2.217988e-03,2.029615e-03,1.955506e-03,1.207442e-03,8.346050e-04,1.329813e-03,1.140636e-03,...,2,137688,3,37665,180,0,2,414,12923,2584
3,N20000001,Road repairs,2.733760e-03,2.985896e-03,3.331585e-03,3.500383e-03,3.506608e-03,3.967531e-03,3.940610e-03,3.570292e-03,...,2,137688,3,37665,180,0,2,414,12923,2584
4,N20000001,Road safety,6.563730e-04,4.837550e-04,3.990420e-04,3.094560e-04,2.172590e-05,-8.859600e-05,-1.572280e-04,-2.813460e-04,...,2,137688,3,37665,180,0,2,414,12923,2584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2780605,S01013481,Excess cold,1.026941e-04,7.528737e-04,5.563542e-04,1.147938e-03,2.770141e-03,1.206036e-03,2.316936e-03,3.127054e-03,...,1,252995,1,24480,96,1,2,298,18643,12357
2780606,S01013481,Excess heat,1.693859e-09,-1.384168e-08,1.343553e-08,-1.013713e-07,-2.880965e-08,-1.337351e-07,1.206930e-08,1.857267e-08,...,1,252995,1,24480,96,1,2,298,18643,12357
2780607,S01013481,Dampness,3.834547e-05,3.763206e-05,1.010562e-05,6.931372e-05,-4.717624e-05,3.616986e-04,7.791065e-06,2.348168e-04,...,1,252995,1,24480,96,1,2,298,18643,12357
2780608,S01013481,Diet change,3.055232e-02,2.467668e-02,6.447578e-02,5.904762e-02,4.999083e-02,6.013978e-02,1.979823e-02,3.186790e-02,...,1,252995,1,24480,96,1,2,298,18643,12357


In [123]:
df.columns

Index(['Lookup_Value', 'co_benefit_type', '2025', '2026', '2027', '2028',
       '2029', '2030', '2031', '2032', '2033', '2034', '2035', '2036', '2037',
       '2038', '2039', '2040', '2041', '2042', '2043', '2044', '2045', '2046',
       '2047', '2048', '2049', '2050', 'scenario', 'total', 'Nation', 'LAD',
       'MSOA', 'LSOA_DZ_CD', 'LSOA_DZ_NM', 'Under_35', 'Over_65', 'EPC',
       'Median_Income', 'Tenure', 'Typology', 'Unemployment', 'Rurality',
       'House_value', 'Fuel_Type', 'Fuel_consumption_total', 'Floor_area',
       'Gas_flag', 'Number_cars', 'Urban_trips', 'Total_vkm', 'Urban_vkm'],
      dtype='object')

In [124]:
# Assuming df is your DataFrame and 'column_name' is the column you're interested in
# df_sampled = df.groupby('LAD').apply(lambda x: x.sample(n=10)).reset_index(drop=True)
df_sampled = df[df.Nation == "NI"]

In [125]:
df_sampled

,Lookup_Value,co_benefit_type,2025,2026,2027,2028,2029,2030,2031,2032,...,Rurality,House_value,Fuel_Type,Fuel_consumption_total,Floor_area,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm
0,N20000001,Air quality,3.495246e-03,4.545631e-03,5.846987e-03,7.227724e-03,8.178627e-03,8.060945e-03,8.426776e-03,8.942410e-03,...,2,137688,3,37665,180,0,2,414,12923,2584
1,N20000001,Noise,1.345039e-03,2.122763e-03,3.191027e-03,3.107497e-03,3.972673e-03,6.072646e-03,6.876487e-03,6.949075e-03,...,2,137688,3,37665,180,0,2,414,12923,2584
2,N20000001,Congestion,2.332261e-03,2.217988e-03,2.029615e-03,1.955506e-03,1.207442e-03,8.346050e-04,1.329813e-03,1.140636e-03,...,2,137688,3,37665,180,0,2,414,12923,2584
3,N20000001,Road repairs,2.733760e-03,2.985896e-03,3.331585e-03,3.500383e-03,3.506608e-03,3.967531e-03,3.940610e-03,3.570292e-03,...,2,137688,3,37665,180,0,2,414,12923,2584
4,N20000001,Road safety,6.563730e-04,4.837550e-04,3.990420e-04,3.094560e-04,2.172590e-05,-8.859600e-05,-1.572280e-04,-2.813460e-04,...,2,137688,3,37665,180,0,2,414,12923,2584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267857,N20003780,Excess cold,9.926216e-05,2.023556e-03,8.575203e-04,-1.099166e-04,-2.065125e-03,4.147600e-03,-1.987975e-04,1.765109e-03,...,2,161985,3,26394,106,0,2,395,12334,2467
2267858,N20003780,Excess heat,-6.201756e-09,-3.063379e-08,-3.561648e-08,-5.411495e-08,-1.781081e-08,-6.124070e-08,-8.144563e-08,-2.712894e-08,...,2,161985,3,26394,106,0,2,395,12334,2467
2267859,N20003780,Dampness,7.290739e-05,-3.292369e-06,-7.793160e-05,2.476983e-04,2.503835e-04,4.885546e-04,1.499853e-04,4.278001e-04,...,2,161985,3,26394,106,0,2,395,12334,2467
2267860,N20003780,Diet change,3.015957e-03,-9.118543e-03,1.803454e-02,1.526367e-02,-3.046252e-02,2.232430e-02,8.573382e-03,6.869538e-03,...,2,161985,3,26394,106,0,2,395,12334,2467


In [126]:
df_sampled[(df_sampled["Lookup_Value"] == "N20000001") & (df_sampled["co_benefit_type"] == "Dampness")][["2028", "scenario"]]

,2028,scenario
9,0.000073,BNZ
556131,-0.000132,Engagement
1112253,0.000131,Tailwinds
1668375,0.000234,Headwinds
2224497,0.000012,Innovation


In [127]:
df_sampled.to_parquet('static/database_onlyIreland.parquet')